In [1]:
%autosave 0

Autosave disabled


In [2]:
import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
# from concorde_class import Concorde
import datetime
import subprocess

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# path for the 'Data' folder provided by DP
dir_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/AOP_DP_Analytics/Data'

In [5]:
region_name = 'Warmsen'

distance_path = dir_loc + '/Instances/' + region_name + '/distances/'

for filename in os.listdir(distance_path):
  print(filename)

distances_31606-13.dat
distances_31606-11.dat
distances_31600-04.dat
distances_31600-03.dat
distances_31600-02.dat
distances_31603-07.dat
distances_31603-08.dat
distances_31604-10.dat
distances_31603-05.dat
distances_31603-06.dat
distances_31606-12.dat
distances_31606-14.dat
distances_31604-09.dat
distances_31600-01.dat


#Functions

In [6]:
# get start and end node for a district
# delete df_day input

def start_end_points(region_name, district):
    # df_day_district = df_day[df_day['district'] == district]

    # get start and end node
    district_path = dir_loc + '/Instances/' + region_name + '/Districts/' + district + '.dat'
    rows_needed = [1]
    file_district = pd.read_csv(district_path, sep='\t', skiprows = lambda x : x not in rows_needed,
                                names=('dum_0', 'start_point', 'end_point', 'dum_1', 'dum_2', 'dum_3', 'dum_4', 'dum_5', 'dum_6')
                                )
    start_point = file_district['start_point'][0]
    end_point = file_district['end_point'][0]

    points = [start_point, end_point]

    return points

In [17]:
# generate original distance matrix and mapping for districts

def ori_generate_distance_matrix_map(region_name, save_loc):
      i = 1

      dir_region = save_loc + '/' + region_name + '_original'
      print(dir_region)
      if not os.path.exists(dir_region):
        os.mkdir(dir_region)
        print('dir_region doesnt exist')

      # dir_region_ori = dir_region + '/original'
      # print(dir_region_ori)
      # if not os.path.exists(dir_region):
      #   os.mkdir(dir_region)
      #   print('dir_ori doesnt exist')

      ori_dm_map_dict_region = {}
      # get distance file and dataframes
      distance_path = dir_loc + '/Instances/' + region_name + '/distances/'
      for filename in os.listdir(distance_path):
        ori_dm_map_dict_district = {}
        district_name = re.sub('.dat', '', filename)
        district_name = re.sub('distances_', '', district_name)

        district_distance = pd.read_csv(distance_path + filename,
                                    names=['pp_1', 'pp_2', 'dist']
                                    )
      
        distance_matrix_df = district_distance.pivot(index = 'pp_1', columns = 'pp_2', values = 'dist')

        # generate and revise start and end nodes
        points = start_end_points(region_name = region_name, district = district_name)
        distance_matrix_df[points[0]][points[1]] = 0
        distance_matrix_df[points[1]][points[0]] = 0
        ori_dm_map_dict_district['start_end_points'] = points


        # distance matrix
        distance_matrix_array = distance_matrix_df.to_numpy()
        distance_matrix_array = distance_matrix_array.tolist()
        ori_dm_map_dict_district['dm'] = distance_matrix_array
        pickle.dump(distance_matrix_array, open(dir_region + '/dm_' + "%s.p"%district_name, "wb"))

        # node mapping
        map_val = list(range(0, len(distance_matrix_df)))
        nodes = distance_matrix_df.index.values.tolist()
        mapping = dict(zip(map_val, nodes))
        ori_dm_map_dict_district['map'] = mapping
        pickle.dump(mapping, open(dir_region + '/map_' + "%s.p"%(district_name), "wb"))

        ori_dm_map_dict_region[district_name] = ori_dm_map_dict_district
        print(i, ':', district_name, ':', len(distance_matrix_array))
        i = i+1

      return ori_dm_map_dict_region

#Generate complete distance matrices

In [18]:
%%time

regions = ['Warmsen', 'Uerze', 'Hannover 92']

all_original = {}

for region in regions:
  ori_dm_map_dict_region = ori_generate_distance_matrix_map(region_name = region, save_loc = '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/')
  all_original[region] = ori_dm_map_dict_region 

/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding//Warmsen_original
1 : 31606-13 : 306
2 : 31606-11 : 349
3 : 31600-04 : 293
4 : 31600-03 : 372
5 : 31600-02 : 417
6 : 31603-07 : 397
7 : 31603-08 : 360
8 : 31604-10 : 380
9 : 31603-05 : 376
10 : 31603-06 : 424
11 : 31606-12 : 420
12 : 31606-14 : 311
13 : 31604-09 : 405
14 : 31600-01 : 427
/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding//Uerze_original
dir_region doesnt exist
1 : 31311-31 : 375
2 : 31311-07 : 288
3 : 31311-04 : 397
4 : 31311-01 : 400
5 : 31311-25 : 398
6 : 31311-22 : 479
7 : 31311-03 : 512
8 : 31311-02 : 416
9 : 31311-24 : 424
10 : 31311-42 : 482
11 : 31311-61 : 513
12 : 31311-51 : 444
13 : 31311-21 : 422
14 : 31311-05 : 421
15 : 31311-62 : 505
16 : 31311-08 : 451
17 : 31311-81 : 490
/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding//Hannover 92_original
dir_region doesnt exist
1 : 30457-02 : 227
2 : 30459-35 : 254
3 : 

In [29]:
name = 'all_original_complete'

pickle.dump(all_original, open('/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results' + '/' + "%s.p"%name, "wb"))

#Check

In [20]:
ori_dm_map_dict_region['30459-32']['start_end_points']

[0, 0]

In [21]:
all_original['Hannover 92']['30459-32'].keys()

dict_keys(['start_end_points', 'dm', 'map'])

In [22]:
ex_dm = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Uerze_original/dm_31311-81.p", "rb"))
ex_map = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Uerze_original/map_31311-81.p", "rb"))

In [23]:
print(ex_map[0])
print(ex_map[1])

0
4


In [24]:
# swap key and value in dict
d_swap = {v: k for k, v in ex_map.items()}

In [25]:
start_point = d_swap[0]
end_point = d_swap[4]

In [26]:
print(ex_dm[start_point][end_point])
print(ex_dm[end_point][start_point])

0.0
0.0


In [27]:
ex_dm_df = pd.DataFrame(ex_dm)
ex_dm_df.head()

,0,1,2,3,4,5,6,7,8,9,...,480,481,482,483,484,485,486,487,488,489
0,0.00,0.00,387.54,388.33,390.68,404.81,397.62,389.43,397.22,401.46,...,369.31,359.44,359.84,378.27,380.78,355.45,304.11,313.20,306.89,854.62
1,0.00,0.00,531.20,531.99,534.34,548.47,541.28,533.09,540.88,545.12,...,512.97,503.10,503.50,521.93,524.44,499.11,447.77,456.86,450.55,998.28
2,396.44,520.60,0.00,45.85,48.20,62.33,55.14,46.95,54.74,58.98,...,100.13,90.26,90.66,109.09,111.60,86.27,149.19,173.81,162.48,728.64
3,398.64,522.80,47.10,0.00,40.11,54.24,47.05,38.86,46.65,50.89,...,102.33,92.46,92.86,111.29,113.80,88.47,151.39,176.01,164.68,730.84
4,400.99,525.15,49.45,41.35,0.00,51.71,44.52,32.88,44.12,48.36,...,104.68,94.81,95.21,113.64,116.15,90.82,153.74,178.36,167.03,733.19


In [28]:
len(all_original['Warmsen']['31600-01']['dm'][0])

427